In [1]:
from glob import glob #retriving an array of files in directories
from tqdm import tqdm
import os
import csv
import random
import pydicom
import numpy as np
import pandas as pd
from skimage import io
from skimage import measure
from skimage.transform import resize
import cv2
import tensorflow as tf
from tensorflow import keras

from matplotlib import pyplot as plt
import matplotlib.patches as patches
import shutil
import pydicom as dicom
from tqdm import tqdm

In [2]:
def make_dirs(out):
    os.makedirs(out)
    os.makedirs(out+"train/")
    os.makedirs(out+"test/")
    os.makedirs(out+"train/NORMAL/")
    os.makedirs(out+"train/PNEUMONIA/")
    os.makedirs(out+"test/NORMAL/")
    os.makedirs(out+"test/PNEUMONIA/")
    os.makedirs(out+"val/NORMAL/")
    os.makedirs(out+"val/PNEUMONIA/")

In [4]:
output_dir = "../dataset/Pneumonia/rsna_data/"
sub = pd.read_csv("../dataset/input/kaggle_rsna/stage_2_train_labels.csv")
detail = pd.read_csv("../dataset/input/kaggle_rsna/stage_2_detailed_class_info.csv")
detail.tail()

,patientId,class
30222,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,Lung Opacity
30223,c1edf42b-5958-47ff-a1e7-4f23d99583ba,Normal
30224,c1f6b555-2eb1-4231-98f6-50a963976431,Normal
30225,c1f7889a-9ea9-4acb-b64c-b737c929599a,Lung Opacity
30226,c1f7889a-9ea9-4acb-b64c-b737c929599a,Lung Opacity


In [7]:
#converting dcm to jpg
folder_path = "../dataset/input/kaggle_rsna/stage_2_train_images/"
images_path = os.listdir(folder_path)
os.makedirs("../dataset/input/kaggle_rsna/stage_2_train_images2/")
jpg_folder_path = "../dataset/input/kaggle_rsna/stage_2_train_images2/"
for n, image in enumerate(images_path):
        ds = dicom.dcmread(os.path.join(folder_path, image))
        rows = []
        pixel_array_numpy = ds.pixel_array
        image = image.replace('.dcm', '.jpg')
        cv2.imwrite(os.path.join(jpg_folder_path, image), pixel_array_numpy)

In [8]:
dis_type = detail["class"]
dis_name = detail["patientId"]
name = sub["patientId"]
value = sub["Target"]
name = [name[i]+".jpg" for i in range(len(name))]
dis_name = [dis_name[i]+".jpg" for i in range(len(dis_name))]
img = glob("../dataset/input/kaggle_rsna/stage_2_train_images2/*") #Getting all images in this folder
filenames = os.listdir("../dataset/input/kaggle_rsna/stage_2_train_images2/")

In [9]:
#list.index(element)
normal,pneumonia = [],[]
normal_name,pneumonia_name = [],[]
for i in tqdm(range(len(filenames))):
    ind = name.index(filenames[i])
    ind1 = dis_name.index(filenames[i])
    if(value[ind] == 0 and dis_type[ind] == "Normal"):
        normal.append(img[i])
        normal_name.append(filenames[i])
    elif(value[ind] == 1):
        pneumonia.append(img[i])
        pneumonia_name.append(filenames[i])

100%|██████████| 26684/26684 [00:15<00:00, 1733.17it/s]


In [11]:
random.shuffle(normal)
random.shuffle(pneumonia)
knn = int(len(normal))
knp = int(len(pneumonia)) 
n1 = int(knn*0.8)
n2 = int(knn*0.9)
p1 = int(knp*0.8)
p2 = int(knp*0.9)
normal_train,normal_test, normal_val = normal[:n1],normal[n1:n2],normal[n2:]
pneumonia_train,pneumonia_test, pneumonia_val = pneumonia[:p1],pneumonia[p1:p2],pneumonia[p2:]

In [12]:
try:
    make_dirs(output_dir)
except:
    pass

In [13]:
for image in normal_train:
    shutil.copy(image,output_dir+"train/NORMAL")
for image in normal_test:
    shutil.copy(image,output_dir+"test/NORMAL")  
for image in normal_val:
    shutil.copy(image,output_dir+"val/NORMAL") 
for image in pneumonia_train:
    shutil.copy(image,output_dir+"train/PNEUMONIA")
for image in pneumonia_test:
    shutil.copy(image,output_dir+"test/PNEUMONIA")
for image in pneumonia_val:
    shutil.copy(image,output_dir+"val/PNEUMONIA")

In [14]:
len(pneumonia_train)

4809